# Preprocesamiento de Datos – Fuentes UNID

Este notebook tiene como objetivo transformar y preparar los datos de las fuentes no identificadas (UNIDs), contenidas en el archivo `unids_3F_beta_err_names.txt`, para su uso en modelos de detección de anomalías como One-Class SVM.

A diferencia de las fuentes ASTRO, las variables de los UNIDs no están logarítmicamente transformadas. Dado que el modelo se entrena con datos ASTRO ya transformados en `log10`, es necesario aplicar la misma transformación a los UNIDs para garantizar la coherencia de escala.

Las variables transformadas serán:
- `E_peak`
- `beta`
- `sigma_det`
- `beta_Rel`

Los datos procesados se guardarán tanto en formato `.csv` como `.txt` para facilitar su reutilización.

In [ ]:
# Transformación Logarítmica y Verificación de Compatibilidad - UNIDs Fermi-LAT
# TFG: Utilización de técnicas de ML a datos del satélite Fermi-Lat para detección de posibles fuentes de materia oscura
# Objetivo: Transformar datos UNIDs a escala logarítmica para compatibilidad con modelo OneClassSVM

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (16, 12)
plt.rcParams['font.size'] = 12

In [ ]:
unids_features = ['E_peak', 'beta', 'sigma_det', 'beta_Rel', 'number']
unids_original = pd.read_csv('../../data/raw/unids_3F_beta_err_names.txt', 
                            sep="\s+", names=unids_features, engine='python', skiprows=1)

print(f"UNIDs cargados: {unids_original.shape}")

print(f"Dimensiones del dataset UNIDs: {unids_original.shape}")
print(f"Número total de fuentes no identificadas: {len(unids_original)}")

In [ ]:
# Ver muestra
unids_original.head()

In [ ]:
unids_original.info()

In [ ]:
# Cargar datos de entrenamiento ASTRO (ya en escala logarítmica)
print("Cargando datos de entrenamiento ASTRO...")
training_features = ['E_peak', 'beta', 'sigma', 'beta_Rel', 'astro_DM']
training_full = pd.read_csv('../../data/raw/XY_bal_log_Rel.txt', 
                           sep="\s+", names=training_features, engine='python', skiprows=1)

# Filtrar solo datos ASTRO (clase 0.0) - datos usados para entrenar OneClassSVM
astro_df = training_full[training_full['astro_DM'] == 0.0].copy()

print(f"- Dataset completo cargado: {training_full.shape}")
print(f"- Datos ASTRO extraídos: {astro_df.shape}")

In [ ]:
astro_df.head()

In [ ]:
astro_df.info()

In [ ]:
print("\nUNIDs (escala lineal):")
unids_original[['E_peak', 'beta', 'sigma_det', 'beta_Rel']].describe()

In [ ]:
print("\nDatos ASTRO - Entrenamiento OneClassSVM (escala logarítmica):")
astro_df[['E_peak', 'beta', 'sigma', 'beta_Rel']].describe()

## Verificaciones

In [ ]:
feature_cols = ['E_peak', 'beta', 'sigma_det', 'beta_Rel']

print("Verificando valores ≤ 0 que causarían problemas con log₁₀:")

problematic_values = {}
for col in feature_cols:
    zero_count = (unids_original[col] == 0).sum()
    negative_count = (unids_original[col] < 0).sum()
    min_positive = unids_original[unids_original[col] > 0][col].min() if (unids_original[col] > 0).any() else None
    
    problematic_values[col] = {
        'zeros': zero_count,
        'negatives': negative_count,
        'min_positive': min_positive,
        'total_problematic': zero_count + negative_count
    }
    
    print(f"\n{col}:")
    print(f"  • Valores = 0: {zero_count}")
    print(f"  • Valores < 0: {negative_count}")
    print(f"  • Valores problemáticos totales: {zero_count + negative_count}")
    print(f"  • Valor positivo mínimo: {min_positive}")
    
    if zero_count + negative_count > 0:
        print(f" ATENCIÓN: {zero_count + negative_count} valores problemáticos")
        # Mostrar algunos ejemplos
        problematic_indices = unids_original[unids_original[col] <= 0].index[:5]
        if len(problematic_indices) > 0:
            print(f"  Primeros índices problemáticos: {problematic_indices.tolist()}")
    else:
        print(f"Sin valores problemáticos")

In [ ]:
# Definir estrategia basada en valores problemáticos
total_problematic = sum([v['total_problematic'] for v in problematic_values.values()])

if total_problematic > 0:
    print(f"Se encontraron {total_problematic} valores problemáticos totales")
    print("\nEstrategia adoptada:")
    print("1. Aplicar transformación log10 a valores positivos")
    print("2. Para valores ≤ 0: aplicar log10 (valor + epsilon) donde epsilon = 1e-10")
    print("3. Verificar que no se introduzcan artifacts")
    
    epsilon = 1e-10
    print(f"\nEpsilon utilizado: {epsilon}")
else:
    print("No hay valores problemáticos")
    print("Estrategia: Aplicar log10 directamente")
    epsilon = 0

## Transformación de las columnas de UNIDS a log(10)

In [ ]:
# Crear copia para transformación
unids_transformed = unids_original.copy()

print("Aplicando transformación log10...")

# Aplicar transformación a cada variable
transformation_log = {}

for col in feature_cols:
    original_col = col
    
    if epsilon > 0:
        # Aplicar epsilon para valores problemáticos
        transformed_values = np.log10(unids_transformed[col] + epsilon)
        transformation_log[col] = f"log10({col} + {epsilon})"
    else:
        # Aplicar log₁₀ directamente
        transformed_values = np.log10(unids_transformed[col])
        transformation_log[col] = f"log10({col})"
    
    unids_transformed[col] = transformed_values
    print(f"- {col}: {transformation_log[col]}")

In [ ]:
print("\nESTADÍSTICAS DESPUÉS DE TRANSFORMACIÓN:")
print("\nUNIDs (escala logarítmica):")
print(unids_transformed[feature_cols].describe())

In [ ]:
print("Comparando rangos entre UNIDs transformados y datos ASTRO (usados para entrenar OneClassSVM):")

compatibility_check = {}
for i, col in enumerate(['E_peak', 'beta', 'sigma_det', 'beta_Rel']):
    astro_col = 'sigma' if col == 'sigma_det' else col
    
    # Rangos
    unid_min, unid_max = unids_transformed[col].min(), unids_transformed[col].max()
    astro_min, astro_max = astro_df[astro_col].min(), astro_df[astro_col].max()
    
    # Verificar superposición
    overlap_min = max(unid_min, astro_min)
    overlap_max = min(unid_max, astro_max)
    has_overlap = overlap_min <= overlap_max
    
    # Verificar si UNIDs están dentro del rango ASTRO
    within_range = (unid_min >= astro_min) and (unid_max <= astro_max)
    
    # Calcular porcentaje de superposición
    if has_overlap:
        overlap_range = overlap_max - overlap_min
        astro_range = astro_max - astro_min
        overlap_percentage = (overlap_range / astro_range) * 100 if astro_range > 0 else 0
    else:
        overlap_percentage = 0
    
    compatibility_check[col] = {
        'unid_range': (unid_min, unid_max),
        'astro_range': (astro_min, astro_max),
        'overlap': has_overlap,
        'within_astro_range': within_range,
        'overlap_range': (overlap_min, overlap_max) if has_overlap else None,
        'overlap_percentage': overlap_percentage
    }
    
    print(f"\n{col} ↔ ASTRO {astro_col}:")
    print(f"  UNIDs transformados: [{unid_min:.3f}, {unid_max:.3f}]")
    print(f"  Datos ASTRO:         [{astro_min:.3f}, {astro_max:.3f}]")
    print(f"  Superposición: {'Sí' if has_overlap else 'No'}")
    print(f"  UNIDs dentro rango ASTRO: {'Sí' if within_range else 'No'}")
    
    if has_overlap:
        print(f"  Rango superposición: [{overlap_min:.3f}, {overlap_max:.3f}]")
        print(f"  % de superposición: {overlap_percentage:.1f}%")
    
    # Advertencias específicas
    if not has_overlap:
        print(f" SIN SUPERPOSICIÓN: Modelo puede no funcionar bien")
    elif overlap_percentage < 50:
        print(f"SUPERPOSICIÓN BAJA: Verificar performance del modelo")
    elif within_range:
        print(f"COMPATIBILIDAD EXCELENTE: UNIDs dentro del espacio ASTRO")


In [ ]:
# Crear visualización antes vs después
fig, axes = plt.subplots(4, 3, figsize=(20, 16))

for i, col in enumerate(feature_cols):
    train_col = 'sigma' if col == 'sigma_det' else col
    
    # Distribución original UNIDs
    axes[i, 0].hist(unids_original[col], bins=50, alpha=0.7, color='lightcoral', 
                   edgecolor='black', label='UNIDs Original')
    axes[i, 0].set_title(f'{col} - Original (Escala Lineal)')
    axes[i, 0].set_xlabel(col)
    axes[i, 0].set_ylabel('Frecuencia')
    axes[i, 0].legend()
    axes[i, 0].grid(True, alpha=0.3)
    
    # Distribución transformada UNIDs
    axes[i, 1].hist(unids_transformed[col], bins=50, alpha=0.7, color='lightgreen', 
                   edgecolor='black', label='UNIDs Transformado')
    axes[i, 1].set_title(f'{col} - Transformado (Log10)')
    axes[i, 1].set_xlabel(f'Log10({col})')
    axes[i, 1].set_ylabel('Frecuencia')
    axes[i, 1].legend()
    axes[i, 1].grid(True, alpha=0.3)
    
    # Comparación superpuesta - UNIDs vs ASTRO solamente
    axes[i, 2].hist(unids_transformed[col], bins=40, alpha=0.6, color='lightgreen', 
                   density=True, label='UNIDs Transform.', edgecolor='black')
    axes[i, 2].hist(astro_df[astro_col], bins=40, alpha=0.6, color='skyblue', 
                   density=True, label='ASTRO (Entrenamiento)', edgecolor='black')
    axes[i, 2].set_title(f'{col} - UNIDs vs ASTRO (OneClassSVM)')
    axes[i, 2].set_xlabel(f'Log10({col})')
    axes[i, 2].set_ylabel('Densidad')
    axes[i, 2].legend()
    axes[i, 2].grid(True, alpha=0.3)

plt.suptitle('Transformación: Original → Log10 → Comparación vs ASTRO (OneClassSVM)', 
             fontsize=16, y=0.98)
plt.tight_layout()
plt.show()


In [ ]:
# Crear visualización de distribuciones
fig, axes = plt.subplots(4, 3, figsize=(20, 16))

for i, col in enumerate(feature_cols):
    astro_col = 'sigma' if col == 'sigma_det' else col
    
    # ANTES: UNIDs original vs ASTRO (escalas diferentes)
    axes[i, 0].hist(unids_original[col], bins=50, alpha=0.7, color='lightcoral', 
                   label=f'UNIDs Original', density=True)
    axes[i, 0].set_title(f'{col} - ANTES: UNIDs Original (Escala Lineal)')
    axes[i, 0].set_xlabel(col)
    axes[i, 0].set_ylabel('Densidad')
    axes[i, 0].legend()
    axes[i, 0].grid(True, alpha=0.3)
    
    # Estadísticas en el gráfico
    mean_orig = unids_original[col].mean()
    std_orig = unids_original[col].std()
    axes[i, 0].text(0.7, 0.8, f'μ = {mean_orig:.2f}\nσ = {std_orig:.2f}', 
                   transform=axes[i, 0].transAxes, bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))
    
    # DESPUÉS: UNIDs transformado
    axes[i, 1].hist(unids_transformed[col], bins=50, alpha=0.7, color='lightgreen', 
                   label=f'UNIDs Transformado', density=True)
    axes[i, 1].set_title(f'{col} - DESPUÉS: UNIDs Log10')
    axes[i, 1].set_xlabel(f'Log10({col})')
    axes[i, 1].set_ylabel('Densidad')
    axes[i, 1].legend()
    axes[i, 1].grid(True, alpha=0.3)
    
    # Estadísticas transformadas
    mean_trans = unids_transformed[col].mean()
    std_trans = unids_transformed[col].std()
    axes[i, 1].text(0.7, 0.8, f'μ = {mean_trans:.2f}\nσ = {std_trans:.2f}', 
                   transform=axes[i, 1].transAxes, bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))
    
    # COMPARACIÓN: UNIDs transformado vs ASTRO (misma escala)
    axes[i, 2].hist(unids_transformed[col], bins=40, alpha=0.6, color='lightgreen', 
                   density=True, label='UNIDs Log10')
    axes[i, 2].hist(astro_df[astro_col], bins=40, alpha=0.6, color='skyblue', 
                   density=True, label='ASTRO')
    axes[i, 2].set_title(f'{col} - COMPARACIÓN: Misma Escala Log10')
    axes[i, 2].set_xlabel(f'Log10({col})')
    axes[i, 2].set_ylabel('Densidad')
    axes[i, 2].legend()
    axes[i, 2].grid(True, alpha=0.3)
    
    # Mostrar superposición
    unid_min, unid_max = unids_transformed[col].min(), unids_transformed[col].max()
    astro_min, astro_max = astro_df[astro_col].min(), astro_df[astro_col].max()
    overlap_min = max(unid_min, astro_min)
    overlap_max = min(unid_max, astro_max)
    
    if overlap_min <= overlap_max:
        axes[i, 2].axvspan(overlap_min, overlap_max, alpha=0.2, color='purple', label='Superposición')
        axes[i, 2].legend()

plt.suptitle('Verificación Visual: Transformación Logarítmica UNIDs', fontsize=18, y=0.98)
plt.tight_layout()
plt.show()


In [ ]:
# Pares de variables más importantes
important_pairs = [
    ('E_peak', 'beta'),
    ('E_peak', 'sigma_det'),
    ('beta', 'sigma_det'),
    ('beta', 'beta_Rel')
]

# ANTES DE LA TRANSFORMACIÓN
fig, axes = plt.subplots(2, 2, figsize=(18, 14))
axes = axes.ravel()

for i, (x_var, y_var) in enumerate(important_pairs):
    astro_x = 'sigma' if x_var == 'sigma_det' else x_var
    astro_y = 'sigma' if y_var == 'sigma_det' else y_var
    
    # UNIDs original (escala lineal)
    axes[i].scatter(unids_original[x_var], unids_original[y_var], 
                   alpha=0.6, c='red', s=20, label='UNIDs Original', marker='o')
    
    # ASTRO (ya en log) - escalar para visualización
    # Nota: No se puede comparar directamente por diferente escala
    axes[i].set_title(f'ANTES: {x_var} vs {y_var}\n(UNIDs lineal - No comparable con ASTRO)')
    axes[i].set_xlabel(x_var)
    axes[i].set_ylabel(y_var)
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.suptitle('Scatter Plots ANTES de Transformación (Escalas Incompatibles)', fontsize=16, y=0.98)
plt.tight_layout()
plt.show()

In [ ]:
# DESPUÉS DE LA TRANSFORMACIÓN
fig, axes = plt.subplots(2, 2, figsize=(18, 14))
axes = axes.ravel()

for i, (x_var, y_var) in enumerate(important_pairs):
    astro_x = 'sigma' if x_var == 'sigma_det' else x_var
    astro_y = 'sigma' if y_var == 'sigma_det' else y_var
    
    # UNIDs transformado
    axes[i].scatter(unids_transformed[x_var], unids_transformed[y_var], 
                   alpha=0.6, c='green', s=25, label='UNIDs Log₁₀', marker='o')
    
    # ASTRO (referencia)
    axes[i].scatter(astro_df[astro_x], astro_df[astro_y], 
                   alpha=0.4, c='blue', s=15, label='ASTRO', marker='^')
    
    axes[i].set_title(f'DESPUÉS: Log10({x_var}) vs Log10({y_var})\n(Escalas Compatibles)')
    axes[i].set_xlabel(f'Log10({x_var})')
    axes[i].set_ylabel(f'Log10({y_var})')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.suptitle('Scatter Plots DESPUÉS de Transformación (Escalas Compatibles)', fontsize=16, y=0.98)
plt.tight_layout()
plt.show()

In [ ]:
# Crear un resumen estadístico visual
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Resumen de medias antes/después
variables = ['E_peak', 'beta', 'sigma_det', 'beta_Rel']
unids_orig_means = [unids_original[col].mean() for col in variables]
unids_trans_means = [unids_transformed[col].mean() for col in variables]
astro_means = [astro_df['sigma' if col == 'sigma_det' else col].mean() for col in variables]

x_pos = np.arange(len(variables))

ax1.bar(x_pos - 0.25, unids_orig_means, 0.25, label='UNIDs Original', color='lightcoral', alpha=0.7)
ax1.bar(x_pos, unids_trans_means, 0.25, label='UNIDs Log10', color='lightgreen', alpha=0.7)
ax1.bar(x_pos + 0.25, astro_means, 0.25, label='ASTRO', color='skyblue', alpha=0.7)
ax1.set_title('Comparación de Medias')
ax1.set_ylabel('Valor Medio')
ax1.set_xlabel('Variables')
ax1.set_xticks(x_pos)
ax1.set_xticklabels(variables, rotation=45)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Resumen de desviaciones estándar
unids_orig_stds = [unids_original[col].std() for col in variables]
unids_trans_stds = [unids_transformed[col].std() for col in variables]
astro_stds = [astro_df['sigma' if col == 'sigma_det' else col].std() for col in variables]

ax2.bar(x_pos - 0.25, unids_orig_stds, 0.25, label='UNIDs Original', color='lightcoral', alpha=0.7)
ax2.bar(x_pos, unids_trans_stds, 0.25, label='UNIDs Log10', color='lightgreen', alpha=0.7)
ax2.bar(x_pos + 0.25, astro_stds, 0.25, label='ASTRO', color='skyblue', alpha=0.7)
ax2.set_title('Comparación de Desviaciones Estándar')
ax2.set_ylabel('Desviación Estándar')
ax2.set_xlabel('Variables')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(variables, rotation=45)
ax2.legend()
ax2.grid(True, alpha=0.3)

# Histograma conjunto de una variable representativa (E_peak)
ax3.hist(unids_original['E_peak'], bins=50, alpha=0.5, color='red', density=True, label='UNIDs Original')
ax3.set_title('E_peak: Distribución Original UNIDs')
ax3.set_xlabel('E_peak (escala lineal)')
ax3.set_ylabel('Densidad')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Histograma después de transformación
ax4.hist(unids_transformed['E_peak'], bins=40, alpha=0.6, color='green', density=True, label='UNIDs Log10')
ax4.hist(astro_df['E_peak'], bins=40, alpha=0.6, color='blue', density=True, label='ASTRO')
ax4.set_title('E_peak: Después de Transformación')
ax4.set_xlabel('Log10(E_peak)')
ax4.set_ylabel('Densidad')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.suptitle('Resumen Visual: Verificación de Transformación Exitosa', fontsize=16, y=0.98)
plt.tight_layout()
plt.show()


In [ ]:
# Renombrar columnas para consistencia
final_column_mapping = {
    'E_peak': 'Log(E_peak)',
    'beta': 'Log(beta)',
    'sigma_det': 'Log(sigma)',
    'beta_Rel': 'Log(beta_Rel)',
    'number': 'number'
}

unids_final = unids_transformed.rename(columns=final_column_mapping)

print("Columnas renombradas para consistencia:")
for old_name, new_name in final_column_mapping.items():
    print(f"  {old_name} → {new_name}")

In [ ]:
print(f"\nDataset final UNIDs transformado:")
print(f"Dimensiones: {unids_final.shape}")
print(f"Columnas: {list(unids_final.columns)}")

In [ ]:
print("\nPrimeras 5 filas del dataset transformado:")
unids_final.head()

In [ ]:
print("\nÚltimas 5 filas del dataset transformado:")
unids_final.tail()

## Generar y guardar archivos con UNIDs transformados

In [ ]:
# Dataset completo transformado (con IDs)
unids_final.to_csv('../../data/processed/unids_log/unids_transformed_complete.csv', index=False)
unids_final.to_csv('../../data/processed/unids_log/unids_transformed_complete.txt', sep='\t', index=False)
print("- unids_transformed_complete.csv / .txt")

# Mapeo de IDs para trazabilidad (POR SI ACASO FALLASE ALGO UN DÍA)
id_mapping = unids_final[['number']].copy()
id_mapping['original_index'] = range(len(id_mapping))
id_mapping.to_csv('../../data/processed/unids_log/unids_transformed_id_mapping.csv', index=False)
print("- unids_transformed_id_mapping.csv")

# Log de transformación aplicada
transformation_log_df = pd.DataFrame([
    {'variable': var, 'transformation': transform, 'epsilon_used': epsilon}
    for var, transform in transformation_log.items()
])
transformation_log_df.to_csv('../../data/processed/unids_log/unids_transformation_log.csv', index=False)
print("- unids_transformation_log.csv")

## Conclusión

Con esta transformación, las fuentes UNID están ahora en la misma escala logarítmica que las fuentes ASTRO. Este paso es esencial para evitar inconsistencias al aplicar modelos de detección de anomalías como One-Class SVM.

Los archivos resultantes (`unids_log.csv` y `unids_log.txt`) están listos para ser utilizados en los notebooks de análisis y predicción posteriores.

## Comparación visual antes y después de la transformación logarítmica

Para comprobar la consistencia entre los datos UNID y ASTRO, se comparan ambas fuentes primero en su escala original y luego tras aplicar la transformación logarítmica base 10 (`log10`) a los UNIDs.

- En escala original, los valores de `E_peak` y `beta` de los UNIDs se encuentran en un rango muy diferente al de los ASTRO.
- Una vez transformados a escala `log10`, los valores de UNIDs se alinean mejor con los de ASTRO, lo que valida la transformación y confirma que ahora los conjuntos son comparables.

Esta transformación es esencial para asegurar la coherencia cuando se utilice el modelo One-Class SVM entrenado exclusivamente con datos ASTRO.